### A tiny spec library in progress for Python


#### What is Spec
`Spec` is a new development (currently alpha) in `Clojure` language which introduces a handy way to specify how the shape of data would look like. Building upon simple predicates, `Spec` lets a composable and re-usable way to specify various components of some data that a program is going to handle. The most basic function of a spec is to check whether given a spec and data, the data `conforms` to the spec, and if not, then explain the reason.

Below we start with a basic superclass called `SpecBase` which has an abstract method `conform` that each spec subclasses implement. We also add few helper functions along the way, in some caes, which repeat the functionality of the classes. But the intension is that the helper functions like `spec`, `conform`, `and_spec`, `or_spec`, `dict_spec` etc. will provide the interface to external modules, while the classes will implement the inner details.

In [276]:
from abc import ABC, abstractmethod

In [292]:
class ConformError(BaseException):
    def __init__(self, reason=None):
        self.reason = reason


class SpecBase(ABC):
    def __init__(self, name):
        self.name = name
    
    @abstractmethod
    def conform(self, data):
        pass
    
    def is_valid(self, data):
        try:
            return True if self.conform(data) else False
        except ConformError as e:
            return False
        except:
            raise
        
    
class SimpleSpec(SpecBase):
    def __init__(self, predicate, name=None):
        super().__init__(name if name else 'spec<{}>'.format(predicate.__name__))   
        self.predicate = predicate
        
    def conform(self, data):
        result = self.predicate(data)
        if result:
            return (True, data, result)
        else:
            raise ConformError((self.name, self.predicate, data, result))

In [293]:
def is_spec_or_predicate(x):
    import collections
    
    return isinstance(x, collections.Callable) or isinstance(x, SpecBase)


def is_spec(x):
    return isinstance(x, SpecBase)


def spec(predicate, name=None):
    from copy import deepcopy
    
    assert is_spec_or_predicate(predicate)
    
    if is_spec(predicate):
        new_spec = deepcopy(predicate)
        new_spec.name = name if name else predicate.name
        return new_spec
    
    return SimpleSpec(predicate, name)

Let us test the definitions created so far.

In [294]:
integer = spec(lambda x: isinstance(x, int), 'integer?')

In [295]:
integer.is_valid(100)

True

In [296]:
integer.is_valid('a')

False

In [297]:
def is_valid(spc, data):
    assert is_spec(spc)
    
    return spc.is_valid(data)

In [298]:
def conform(spc, data):
    assert is_spec(spc)
    
    return spc.conform(data)

In [299]:
conform(integer, 100)

(True, 100, True)

In [300]:
conform(integer, None)

ConformError: ('integer?', <function <lambda> at 0x10df8c488>, None, False)

We now create two basic ways to `compose` specs:
* create an `and` relation among specs
* create an `or` relation among specs

In [301]:
class AndSpec(SpecBase):
    def __init__(self, specs, name=None):
        self.specs = specs
        for s in self.specs:
            assert is_spec(s)
        self.name = 'AndSpec' if not name else name
        
    def conform(self, data):
        conf_result = None
        
        for s in self.specs:
            conf_result = s.conform(data)
            
        return conf_result
     
    
class OrSpec(SpecBase):
    def __init__(self, specs, name=None):
        self.specs = specs
        for s in self.specs:
            assert is_spec(s)
        self.name = 'OrSpec' if not name else name
        
    def conform(self, data):
        conformed = False
        conf_result = None
        
        for s in self.specs:
            if isinstance(s, SpecBase):
                try:
                    conf_result = s.conform(data)
                    conformed = True
                except:
                    conformed = False
            
            if conformed:
                return conf_result
        
        raise ConformError((self.name, self.specs, data, None))

In [302]:
def and_spec(*spc, **kargs):
    return AndSpec(spc, **kargs)


def or_spec(*spc, **kargs):
    return OrSpec(spc, **kargs)

Some testing code follows.

In [303]:
string = spec(lambda x: isinstance(x, str), 'string?')

In [304]:
string10 = and_spec(string, spec(lambda x: len(x) < 10, 'len<10?'), name='string_with_len_10?')

In [305]:
conform(string10, 'hola')

(True, 'hola', True)

In [306]:
is_valid(string10, 11)

False

In [307]:
is_valid(string10, 'xxxxxxxxzxxcxcxcxcxccxcxccxcxcx')

False

In [308]:
sp3 = or_spec(integer, string10)

In [309]:
is_valid(sp3, 1)

True

In [310]:
is_valid(sp3, 'aaaaa')

True

In [311]:
is_valid(sp3, 'sdfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdffds')

False

In [312]:
conform(sp3, 3.4)

ConformError: ('OrSpec', (<__main__.SimpleSpec object at 0x10df80e48>, <__main__.AndSpec object at 0x10ded3f28>), 3.4, None)

In [313]:
conform(string10, 90)

ConformError: ('string?', <function <lambda> at 0x10df24b70>, 90, False)

Now we add the spec for a `dict`. One thing to note here is that - we could either specify the requirements for a `dict` by its required and optional keys, _or_ we could specify what all keys and corresponding values would look like. Similar cases are handled for `list` as well.

In [331]:
class DictSpec(SpecBase):
    def __init__(self, req=None, opt=None, allow_more=True, name=None):
        self.name = 'DictSpec' if not name else name
        self.req_specs = dict() if not req else req
        self.opt_specs = dict() if not opt else opt
        self.allow_more = allow_more
        
    def conform(self, data):
        if not isinstance(data, dict):
            raise ConformError((self.name, dict, data, None))
            
        for rk in self.req_specs.keys():
            assert is_spec(self.req_specs[rk])
            
            val = data.get(rk, None)
            if val is None:
                raise ConformError(('required_key {}'.format(rk), self.req_specs[rk], data, None))
                
            self.req_specs[rk].conform(val)
        
        for rk in self.opt_specs.keys():
            assert is_spec(self.opt_specs[rk])
            
            val = data.get(rk, None)
            
            if val:
                self.opt_specs[rk].conform(val)
            
        if not self.allow_more:
            total_keys = set(list(self.req_specs.keys()) + list(self.opt_specs.keys()))
            all_keys = set(data.keys())
            
            rem_key_count = len(all_keys - total_keys)
            
            if rem_key_count > 0:
                raise ConformError(('more_data', self, data, None))
                
        return (True, data, None)
    
    
class DictSimpleSpec(SpecBase):
    def __init__(self, key_spec, val_spec, name=None):
        assert is_spec(key_spec)
        assert is_spec(val_spec)
        
        self.name = 'DictSimpleSpec' if not name else name
        self.key_spec = key_spec
        self.val_spec = val_spec
        
    def conform(self, data):
        if not isinstance(data, dict):
            raise ConformError((self.name, dict, data, None))
            
        for k in data.keys():
            self.key_spec.conform(k)
            self.val_spec.conform(data[k])
            
        return (True, data, None)
    

def dict_spec(req=None, opt=None, name=None, allow_more=True, kind=None):
    if kind and isinstance(kind, tuple):
        key_spec = kind[0]
        val_spec = kind[1]
        
        return DictSimpleSpec(key_spec, val_spec, name=name)
    
    return DictSpec(req=req, opt=opt, name=name, allow_more=allow_more)

Testing code follows.

In [332]:
first_name = spec(string10, 'first_name')

In [333]:
last_name = and_spec(string, spec(lambda x: len(x) <= 35, 'len<35'), name='last_name')

In [334]:
age = spec(integer)

In [335]:
person = dict_spec(req=dict(fname=first_name, lname=last_name), opt=dict(age=age))

In [336]:
conform(person, {'fname': 'Sourav', 'lname': 'Datta'})

(True, {'fname': 'Sourav', 'lname': 'Datta'}, None)

In [337]:
conform(person, {'fname': 'Sourav'})

ConformError: ('required_key lname', <__main__.AndSpec object at 0x10df41898>, {'fname': 'Sourav'}, None)

In [338]:
conform(person, {'lname': 'sdfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsffdsfdsfsfdsfdsfdsfdsfsdfdsfdsfdsfdsffdsfdsfds'})

ConformError: ('required_key fname', <__main__.AndSpec object at 0x10df73c88>, {'lname': 'sdfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsffdsfdsfsfdsfdsfdsfdsfsdfdsfdsfdsfdsffdsfdsfds'}, None)

In [339]:
conform(person, {'fname': 'Ok', 'lname': 'sdfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsffdsfdsfsfdsfdsfdsfdsfsdfdsfdsfdsfdsffdsfdsfds'})

ConformError: ('len<35', <function <lambda> at 0x10df96510>, 'sdfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsffdsfdsfsfdsfdsfdsfdsfsdfdsfdsfdsfdsffdsfdsfds', False)

In [340]:
conform(person, {'fname': 'Sourav', 'lname': 'Datta', 'age': '35 years'})

ConformError: ('integer?', <function <lambda> at 0x10df8c488>, '35 years', False)

In [344]:
person = dict_spec(req=dict(fname=first_name, lname=last_name), opt=dict(age=age), name='person', allow_more=False)

In [345]:
conform(person, {'fname': 'Sourav', 'lname': 'Datta', 'age': 35})

(True, {'age': 35, 'fname': 'Sourav', 'lname': 'Datta'}, None)

In [346]:
conform(person, {'fname': 'Sourav', 'lname': 'Datta', 'age': 35, 'address': 'Bristol'})

ConformError: ('more_data', <__main__.DictSpec object at 0x10df22f60>, {'fname': 'Sourav', 'lname': 'Datta', 'age': 35, 'address': 'Bristol'}, None)

In [347]:
name_count = dict_spec(kind=(string, integer), name='name_count')

In [348]:
conform(name_count, {'name': 11, 'age': 34})

(True, {'age': 34, 'name': 11}, None)

In [350]:
conform(name_count, {'name': 11, 'age': 34.0})

ConformError: ('integer?', <function <lambda> at 0x10df8c488>, 34.0, False)

In a similar manner, here is a spec for `list`.

In [353]:
class ListSpec(SpecBase):
    def __init__(self, spec_list, name=None):
        self.name = 'ListSpec' if not name else name
        
        if isinstance(spec_list, list):
            for s in spec_list:
                assert is_spec(s)
        else:
            assert is_spec(spec_list)
            
        self.spec_list = spec_list
        
    def conform(self, data):
        if not isinstance(data, list):
            raise ConformError((self.name, list, data, None))
            
        if isinstance(self.spec_list, list):
            if len(self.spec_list) != len(data):
                raise ConformError((self.name, len, data, None))
            
            for i in range(len(data)):
                self.spec_list[i].conform(data[i])
        else:
            for d in data:
                self.spec_list.conform(d)
                
        return (True, data, None)

def list_spec(spec_list, name=None):
    return ListSpec(spec_list, name=name)

Test code.

In [354]:
string_list = list_spec(string, name='list of strings')

In [355]:
conform(string_list, ['a', 'b', 'c'])

(True, ['a', 'b', 'c'], None)

In [356]:
conform(string_list, [])

(True, [], None)

In [357]:
conform(string_list, ['a', 3])

ConformError: ('string?', <function <lambda> at 0x10df24b70>, 3, False)

In [359]:
info_list = list_spec([string, integer, person], 'info')

In [360]:
conform(info_list, ['Person', 1001, dict(fname='Arya', age=14)])

ConformError: ('required_key lname', <__main__.AndSpec object at 0x10df41898>, {'fname': 'Arya', 'age': 14}, None)

In [361]:
conform(info_list, ['Person', 1001, dict(fname='Arya', age=14, lname='Stark')])

(True, ['Person', 1001, {'age': 14, 'fname': 'Arya', 'lname': 'Stark'}], None)

In [362]:
conform(info_list, ['Person', dict(fname='Arya', lname='Stark'), 1001])

ConformError: ('integer?', <function <lambda> at 0x10df8c488>, {'fname': 'Arya', 'lname': 'Stark'}, False)

Spec for `tuple`.

In [377]:
class TupleSpec(SpecBase):
    def __init__(self, spec_list, name=None):
        self.name = 'TupleSpec' if not name else name
        
        assert isinstance(spec_list, list) or isinstance(spec_list, tuple)
        
        if isinstance(spec_list, list):
            for s in spec_list:
                assert is_spec(s)
            
        self.spec_list = spec_list
        
    def conform(self, data):
        if not isinstance(data, tuple):
            raise ConformError((self.name, tuple, data, None))
            
        if len(self.spec_list) != len(data):
            raise ConformError((self.name, len, data, None))
            
        for i in range(len(data)):
            self.spec_list[i].conform(data[i])
        
        return (True, data, None)

def tuple_spec(spec_list, name=None):
    return TupleSpec(spec_list, name=name)

Test code.

In [378]:
real = spec(lambda x: isinstance(x, float), 'real?')

In [379]:
coords = tuple_spec((real, real, real))

In [380]:
conform(coords, (2.3, 12.4, 0.56))

(True, (2.3, 12.4, 0.56), None)

In [381]:
conform(coords, [2.3, 12.4, 0.56])

ConformError: ('TupleSpec', <class 'tuple'>, [2.3, 12.4, 0.56], None)

In [382]:
conform(coords, ('2.3', 12.4, 0.56))

ConformError: ('real?', <function <lambda> at 0x10df678c8>, '2.3', False)

In [383]:
conform(real, 4.5)

(True, 4.5, True)